In [ ]:
%pip install PyCryptodome

In [ ]:
from Crypto.Util.number import inverse, long_to_bytes
import math


In [ ]:
dot_prod = lambda v1, v2: sum(x*y for x,y in zip(v1,v2))
norm = lambda v: math.sqrt(dot_prod(v,v))

#taken from previous challenge
def gaussian_reduction(v1: tuple[int,int], v2: tuple[int,int]) -> tuple[int,int]:
    while True:
        if norm(v1) > norm(v2):
            v1, v2 = v2, v1 #swap vectors
        m = int(round(dot_prod(v1,v2) / dot_prod(v1,v1)))
        if m == 0: break
        v2 = tuple(v2[i] - m*v1[i] for i in range(len(v1)))
    return v1, v2

Let's try to understand source.py:

```python
def gen_key():
    q = getPrime(512)
    upper_bound = int(math.sqrt(q // 2))
    lower_bound = int(math.sqrt(q // 4))
    f = random.randint(2, upper_bound)
    while True:
        g = random.randint(lower_bound, upper_bound)
        if math.gcd(f, g) == 1:
            break
    h = (inverse(f, q)*g) % q
    return (q, h), (f, g)
``` 

Let's write the code in mathematical notation. We generate some prime $q$ and two co-prime numbers $f$ and $g$. We define $h$ in so:
$$f * h = g \pmod q$$

To encrypt a message $m$ we do the following $e = (h*r + m) \pmod q$. To decrypt we calculate:
$$m = (f*e \pmod q) * f^{-1} \pmod g$$
where $f^{-1}$ is the inverse of $f$ modulo $g$.

Why does it works?
$$(f*e \pmod q) * f^{-1} \pmod g = (f*h*r + f*m \pmod q) * f^{-1} \pmod g $$
now remember how we defined h and we get:
$$(g*r + f*m \pmod q) * f^{-1} \pmod g$$
now because $m, f, g, r < \sqrt{\frac{q}{2}}$ every mutipication of a pair of them is less than $q$:
$$\begin{aligned}
(g*r \pmod q) * f^{-1}\pmod g &= g*r*f^{-1}\pmod g = 0 \\
(f*m \pmod q) * f^{-1} \pmod g &= f*m*f^{-1} \pmod g = m
\end{aligned}$$

Now... how can we use our knowledge of lattices to decrypt the flag?

Let's take a look at the definition of $h$ again:
$$ g = f * h \pmod q$$
We can rewrite it as:
$$g = f * h - c * q$$
where $c$ is some integer. The trick here is to add some trivail equation to the system of equations:
$$f = f * 1 - c * 0$$
so we can combine them:
$$(g,f) = f*(h,1) - c*(q,0)$$
Which is great! It means the lattice created by $(q,0)$ and $(h,1)$ contains $(g,f)$. If $(g,f)$ is shorter than $(q,0)$ and $(h,1)$ we can use gaussian reduction to find it.

In [ ]:
q, h = (7638232120454925879231554234011842347641017888219021175304217358715878636183252433454896490677496516149889316745664606749499241420160898019203925115292257, 2163268902194560093843693572170199707501787797497998463462129592239973581462651622978282637513865274199374452805292639586264791317439029535926401109074800)
e = 5605696495253720664142881956908624307570671858477482119657436163663663844731169035682344974286379049123733356009125671924280312532755241162267269123486523

In [ ]:
v1 = (q,0)
v2 = (h,1)
u1, u2 = gaussian_reduction(v2,v1)
u1, u2

In [ ]:
g, f = u1 #as it's elements are positive

In [ ]:
#taken from source.py
def decrypt(q, h, f, g, e):
    a = (f*e) % q
    m = (a*inverse(f, g)) % g
    return m

long_to_bytes(decrypt(q, h, f, g, e))